In [1]:
!pip install datasets evaluate transformers[sentencepiece] peft tqdm 
!pip install scikit-learn

!pip install --upgrade peft


[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"
base_model = "meta-llama/Llama-2-7b-hf"
peft_model = "FinGPT/fingpt-mt_llama2-7b_lora"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, device_map=device, max_length=512)

model = PeftModel.from_pretrained(model, peft_model)

model = model.eval()

c:\Users\Carlos\Documents\git\llms\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.43s/it]


In [2]:
from fiqa import test_fiqa

dataset = test_fiqa(model, tokenizer)



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: @gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?
Answer: 


Total len: 2. Batchsize: 8. Total steps: 1


100%|██████████| 1/1 [00:11<00:00, 11.03s/it]

Context: ['<s> Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: $HCP Come to the party and buy this -gonna give solid gains and a dividend $$$$$$\nAnswer: </s></s></s></s></s></s></s></s></s></s></s></s></s> No</s>', '<s> Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: @gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?\nAnswer:  negative</s>']
Acc: 0.5. F1 macro: 0.3333333333333333. F1 micro: 0.5. F1 weighted (BloombergGPT): 0.5. 


In [4]:
dataset.to_csv('results/llama2-7b-fingpt.csv')

In [3]:
import pandas as pd
dataset#.query('new_out == 1')
pd.set_option('display.max_colwidth', None)
dataset["out_text"]
dataset

,input,output,instruction,context,target,out_text,new_out
0,$HCP Come to the party and buy this -gonna give solid gains and a dividend $$$$$$,0,What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.,Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: $HCP Come to the party and buy this -gonna give solid gains and a dividend $$$$$$\nAnswer:,0,</s></s></s></s></s></s></s></s></s></s></s></s></s> No</s>,1
1,@gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?,2,What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.,Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: @gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?\nAnswer:,2,negative</s>,2


In [4]:
instruction = """Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: France raises concerns over proposed LSE-Deutsche Boerse deal
Answer: """


print(instruction)

model_input = tokenizer(instruction, return_tensors="pt")
model_input = model_input.to(device)

res = model.generate(**model_input)
decoded = tokenizer.batch_decode(res)
decoded

Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: France raises concerns over proposed LSE-Deutsche Boerse deal
Answer: 


['<s> Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.\nInput: France raises concerns over proposed LSE-Deutsche Boerse deal\nAnswer:  negative</s>']